In [0]:
CREATE WIDGET TEXT catalog DEFAULT "";

In [0]:
USE CATALOG identifier(:catalog);

-- DROP SCHEMA IF EXISTS moviebuster_gold CASCADE;
-- CREATE SCHEMA moviebuster_gold;

USE SCHEMA moviebuster_gold;

In [0]:
CREATE OR REPLACE TABLE dim_customer (
    customer_id INT NOT NULL,
    first_name STRING,
    last_name STRING,
    email STRING,
    active BOOLEAN,
    create_date DATE,
    address STRING,
    postal_code STRING,
    city STRING,
    country STRING,
    CONSTRAINT pk_customer PRIMARY KEY (customer_id)
);

INSERT INTO dim_customer
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    c.active,
    c.create_date,
    a.address,
    a.postal_code,
    ci.city,
    co.country
FROM identifier(:catalog || '.moviebuster_bronze.customer') c
JOIN identifier(:catalog || '.moviebuster_bronze.address') a ON c.address_id = a.address_id
JOIN identifier(:catalog || '.moviebuster_bronze.city') ci ON a.city_id = ci.city_id
JOIN identifier(:catalog || '.moviebuster_bronze.country') co ON ci.country_id = co.country_id;

In [0]:
CREATE OR REPLACE TABLE dim_staff(
  staff_id INT NOT NULL,
  first_name STRING,
  last_name STRING,
  email STRING,
  active BOOLEAN,
  username STRING,
  address STRING,
  postal_code STRING,
  city STRING,
  country STRING,
  staff_store_id INT,
  CONSTRAINT pk_staff PRIMARY KEY (staff_id)
);

INSERT INTO dim_staff
  SELECT
    s.staff_id,
    s.first_name,
    s.last_name,
    s.email,
    s.active,
    s.username,
    a.address,
    a.postal_code,
    ci.city,
    co.country,
    st.store_id AS staff_store_id
  FROM
    identifier(:catalog || '.moviebuster_bronze.staff') s
      JOIN identifier(:catalog || '.moviebuster_bronze.address') a
        ON s.address_id = a.address_id
      JOIN identifier(:catalog || '.moviebuster_bronze.city') ci
        ON a.city_id = ci.city_id
      JOIN identifier(:catalog || '.moviebuster_bronze.country') co
        ON ci.country_id = co.country_id
      JOIN identifier(:catalog || '.moviebuster_bronze.store') st
        ON s.store_id = st.store_id;

In [0]:
CREATE OR REPLACE TABLE dim_film(
  film_id INT NOT NULL,
  title STRING,
  release_year INT,
  rating STRING,
  rental_duration INT,
  rental_rate DECIMAL(5, 2),
  replacement_cost DECIMAL(5, 2),
  special_features STRING,
  language STRING,
  categories STRING,
  CONSTRAINT pk_film PRIMARY KEY (film_id)
);

INSERT INTO dim_film
  SELECT
    f.film_id,
    f.title,
    f.release_year,
    f.rating,
    f.rental_duration,
    f.rental_rate,
    f.replacement_cost,
    f.special_features,
    l.name AS language,
    STRING_AGG(DISTINCT cat.name) AS categories
  FROM
    identifier(:catalog || '.moviebuster_bronze.film') f
      JOIN identifier(:catalog || '.moviebuster_bronze.language') l
        ON f.language_id = l.language_id
      LEFT JOIN identifier(:catalog || '.moviebuster_bronze.film_category') fc
        ON f.film_id = fc.film_id
      LEFT JOIN identifier(:catalog || '.moviebuster_bronze.category') cat
        ON fc.category_id = cat.category_id
  GROUP BY
    f.film_id,
    f.title,
    f.release_year,
    f.rating,
    f.rental_duration,
    f.rental_rate,
    f.replacement_cost,
    f.special_features,
    l.name;

In [0]:
CREATE OR REPLACE TABLE dim_store(
  store_id INT NOT NULL,
  manager_staff_id INT,
  address STRING,
  postal_code STRING,
  city STRING,
  country STRING,
  CONSTRAINT pk_store PRIMARY KEY (store_id)
);

INSERT INTO dim_store
  SELECT
    st.store_id,
    st.manager_staff_id,
    a.address,
    a.postal_code,
    ci.city,
    co.country
  FROM
    identifier(:catalog || '.moviebuster_bronze.store') st
      JOIN identifier(:catalog || '.moviebuster_bronze.address') a
        ON st.address_id = a.address_id
      JOIN identifier(:catalog || '.moviebuster_bronze.city') ci
        ON a.city_id = ci.city_id
      JOIN identifier(:catalog || '.moviebuster_bronze.country') co
        ON ci.country_id = co.country_id;

In [0]:
CREATE OR REPLACE TABLE rental_fact (
    rental_id INT NOT NULL,
    rental_date DATE,
    return_date DATE,
    rental_duration INT,
    amount DECIMAL(10,2),

    -- Foreign Keys to dimension tables
    customer_id INT,
    staff_id INT,
    film_id INT,
    store_id INT,

    -- Primary Key
    CONSTRAINT pk_rental_fact PRIMARY KEY (rental_id),

    -- Foreign Key Constraints (informational)
    CONSTRAINT fk_rental_customer FOREIGN KEY (customer_id) REFERENCES dim_customer (customer_id),
    CONSTRAINT fk_rental_staff FOREIGN KEY (staff_id) REFERENCES dim_staff (staff_id),
    CONSTRAINT fk_rental_film FOREIGN KEY (film_id) REFERENCES dim_film (film_id),
    CONSTRAINT fk_rental_store FOREIGN KEY (store_id) REFERENCES dim_store (store_id)
);

INSERT INTO rental_fact
SELECT
    r.rental_id,
    r.rental_date,
    r.return_date,
    DATEDIFF(r.return_date, r.rental_date) AS rental_duration,
    p.amount,
    r.customer_id,
    r.staff_id,
    i.film_id,
    i.store_id
FROM identifier(:catalog || '.moviebuster_bronze.rental') r
JOIN identifier(:catalog || '.moviebuster_bronze.payment') p ON r.rental_id = p.rental_id
JOIN identifier(:catalog || '.moviebuster_bronze.inventory') i ON r.inventory_id = i.inventory_id;

In [0]:
CREATE OR REPLACE VIEW rental_fact_view AS
SELECT
  -- Fact
  f.rental_id,
  f.rental_date,
  f.return_date,
  f.rental_duration,
  f.amount,
  -- Customer Dimension
  dc.customer_id,
  dc.first_name AS customer_first_name,
  dc.last_name AS customer_last_name,
  dc.email AS customer_email,
  dc.active AS customer_active,
  dc.address AS customer_address,
  dc.postal_code AS customer_postal_code,
  dc.city AS customer_city,
  dc.country AS customer_country,
  -- Staff Dimension
  ds.staff_id,
  ds.first_name AS staff_first_name,
  ds.last_name AS staff_last_name,
  ds.email AS staff_email,
  ds.username AS staff_username,
  ds.active AS staff_active,
  ds.address AS staff_address,
  ds.postal_code AS staff_postal_code,
  ds.city AS staff_city,
  ds.country AS staff_country,
  ds.staff_store_id,
  -- Film Dimension
  df.film_id,
  df.title AS film_title,
  df.release_year,
  df.rating,
  df.rental_duration AS film_rental_duration,
  df.rental_rate,
  df.replacement_cost,
  df.special_features,
  df.language,
  df.categories,
  -- Store Dimension
  dstore.store_id,
  dstore.manager_staff_id,
  dstore.address AS store_address,
  dstore.postal_code AS store_postal_code,
  dstore.city AS store_city,
  dstore.country AS store_country
FROM
  rental_fact f
    JOIN dim_customer dc
      ON f.customer_id = dc.customer_id
    JOIN dim_staff ds
      ON f.staff_id = ds.staff_id
    JOIN dim_film df
      ON f.film_id = df.film_id
    JOIN dim_store dstore
      ON f.store_id = dstore.store_id
WHERE
  YEAR(f.rental_date) = '2005';

## ABAC

In [0]:
CREATE OR REPLACE FUNCTION mask_email(email STRING)
RETURNS STRING
RETURN regexp_replace(email, '(.*)@', '');

In [0]:
CREATE OR REPLACE FUNCTION mask_password(password STRING)
RETURNS STRING
RETURN '*****';

In [0]:
CREATE OR REPLACE FUNCTION mask_address(password STRING)
RETURNS STRING
RETURN '*****';

In [0]:
CREATE OR REPLACE POLICY mask_emails
ON SCHEMA moviebuster_gold
COLUMN MASK mask_email
TO `account users`
FOR TABLES
MATCH COLUMNS hasTag('class.email_address') AS email
ON COLUMN email;

In [0]:
CREATE OR REPLACE POLICY mask_addresses
ON SCHEMA moviebuster_gold
COLUMN MASK mask_address
TO `account users`
FOR TABLES
MATCH COLUMNS hasTag('class.location') AS email
ON COLUMN email;